In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

import scsavailability as scs
    
import statsmodels.api as sm
from scipy import stats

from scsavailability import features as feat, model as md, plotting as pt, results as rs

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold

In [2]:
path = 'C:/Users/Jamie.williams/OneDrive - Newton Europe Ltd/Castle Donnington/Data/'

In [3]:
at = pd.read_csv(path+'active_totes_20201210.csv')
av = pd.read_csv(path+'Availability_with_Grey&Blue_1811-0912.csv',names = ["timestamp","Pick Station","Availability","Blue Tote Loss","Grey Tote Loss"])
#fa = pd.read_csv(path + 'scs_tote_matched.csv')
scs_raw = pd.read_csv(path + 'Faults20_11-10_12.csv')

In [4]:
fa = feat.add_code(scs_raw)
fa, unmapped = feat.add_tote_colour(fa)

Running: Function "load_tote_lookup" (id=2668445527488) was called 1 times


2021-01-04 19:46:37,469 - NumExpr defaulting to 8 threads.


In [5]:
at = feat.pre_process_AT(at)

Running: Function "pre_process_AT" (id=2668445525904) was called 1 times


In [6]:
av = feat.pre_process_av(av)

Running: Function "pre_process_av" (id=2668445526768) was called 1 times


In [7]:
fa = feat.preprocess_faults(fa,remove_same_location_faults = True)

Running: Function "preprocess_faults" (id=2668445527920) was called 1 times
Running: Function "load_module_lookup" (id=2668442696432) was called 1 times
Running: Function "load_ID_lookup" (id=2668445524176) was called 1 times
duplicated location faults removed - max duration kept
HOTFIX: Quadrant only faults, PTT Asset Code update


In [8]:
fa_floor = feat.floor_shift_time_fa(fa, shift=0)

C:\Users\Jamie.williams\Anaconda3\lib\site-packages\scsavailability\features.py:288: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fa_floor['Duration'][i] = fa_floor['Time Left'][i]
C:\Users\Jamie.williams\Anaconda3\lib\site-packages\scsavailability\features.py:289: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fa_floor['Duration'][i+1] = fa_floor['Duration'][i+1] - fa_floor['Duration'][i]
C:\Users\Jamie.williams\Anaconda3\lib\site-packages\scsavailability\features.py:285: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pyd

In [18]:
df,fa_PTT = feat.create_PTT_df(fa_floor,at,av)

Running: Function "load_PTT_lookup" (id=2668445526192) was called 37 times
Previous Hours Weighted
Running: Function "aggregate_availability" (id=2668445527344) was called 37 times
Running: Function "load_PTT_lookup" (id=2668445526192) was called 38 times
Previous Hours Weighted
Running: Function "aggregate_availability" (id=2668445527344) was called 38 times
Running: Function "load_PTT_lookup" (id=2668445526192) was called 39 times
Previous Hours Weighted
Running: Function "aggregate_availability" (id=2668445527344) was called 39 times
Running: Function "load_PTT_lookup" (id=2668445526192) was called 40 times
Previous Hours Weighted
Running: Function "aggregate_availability" (id=2668445527344) was called 40 times
Running: Function "load_PTT_lookup" (id=2668445526192) was called 41 times
Previous Hours Weighted
Running: Function "aggregate_availability" (id=2668445527344) was called 41 times
Running: Function "load_PTT_lookup" (id=2668445526192) was called 42 times
Previous Hours Weigh

In [10]:
X,y = md.gen_feat_var(df,target = 'Downtime', features = ['Totes','Faults'])
y=1-y

In [31]:
X_train, X_test, y_train, y_test = md.split(X,y,split_options = {'test_size': 0.3,
                                                                 'random_state': None})


R2_cv,R2_OOS,Coeff = md.run_OLS(X_train = X_train,y_train = y_train,X_test = X_test,y_test=y_test, n = 30)

C:\Users\Jamie.williams\Anaconda3\lib\site-packages\statsmodels\base\model.py:1362: RuntimeWarning: invalid value encountered in true_divide
  return self.params / self.bse
C:\Users\Jamie.williams\Anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:1932: RuntimeWarning: invalid value encountered in less_equal
  cond2 = cond0 & (x <= _a)



Cross Validation Scores: 
 
        R2 Scores
1       0.453725
2       0.439019
3       0.492484
4       0.502838
5       0.485948
Mean    0.474803
STD     0.024280
292
                                 OLS Regression Results                                
Dep. Variable:               Downtime   R-squared (uncentered):                   0.958
Model:                            OLS   Adj. R-squared (uncentered):              0.957
Method:                 Least Squares   F-statistic:                              919.2
Date:                Mon, 04 Jan 2021   Prob (F-statistic):                        0.00
Time:                        19:57:50   Log-Likelihood:                          6868.6
No. Observations:               11899   AIC:                                 -1.316e+04
Df Residuals:                   11611   BIC:                                 -1.103e+04
Df Model:                         288                                                  
Covariance Type:            nonrobust 

C:\Users\Jamie.williams\Anaconda3\lib\site-packages\statsmodels\base\model.py:1362: RuntimeWarning: invalid value encountered in true_divide
  return self.params / self.bse
C:\Users\Jamie.williams\Anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:1932: RuntimeWarning: invalid value encountered in less_equal
  cond2 = cond0 & (x <= _a)


In [32]:
R2_OOS

0.4323306570410629

In [19]:
Output = rs.create_output(fa_PTT,Coeff)

In [30]:
df

,timestamp,Downtime,Blue Tote Loss,Grey Tote Loss,C05,C0501ACH011,C0501RDC004,C0501STA002,C0501STA012,C0501STA020,...,SCS202,SCS203,SCS204,SCS205,SCS206,SCSM20,C1404RDC146,C1404STA149,C14PTT202,TOTES
0,2020-11-20 00:00:00,0.183832,0.183832,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,36.220339
1,2020-11-20 01:00:00,0.193668,0.193668,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,32.550000
2,2020-11-20 02:00:00,0.685402,0.572000,0.113402,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,23.633333
3,2020-11-20 03:00:00,0.032177,0.032177,0.000000,0.0,0.0,0.0,0.0,7.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,33.533333
4,2020-11-20 04:00:00,0.504636,0.504636,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,32.933333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
469,2020-12-09 13:00:00,0.010749,0.006667,0.004083,0.0,0.0,0.0,0.0,0.0,0.0,...,342.0,168.0,0.0,428.0,0.0,37.0,0.0,0.0,0.0,49.266667
470,2020-12-09 14:00:00,0.115997,0.000000,0.115997,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,305.0,288.0,1852.0,0.0,128.0,0.0,0.0,0.0,59.266667
471,2020-12-09 15:00:00,0.070237,0.004000,0.066237,0.0,0.0,0.0,0.0,0.0,0.0,...,576.0,0.0,208.0,1102.0,0.0,0.0,0.0,451.0,109.0,53.666667
472,2020-12-09 16:00:00,0.023455,0.001333,0.022121,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,546.0,183.0,0.0,0.0,0.0,0.0,5.0,52.366667
